# Flickr30k dataset analysation

### 0. Library and modules imports

In [3]:
import datacleaner
import pandas as pd

1. Load the dataset (we will add a proper download link once the dataset ready)

In [12]:
df = pd.read_csv('archive/flickr30k_images/results.csv', sep = '|')

In [13]:
df.head()

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


### 2. Dataset analysation 

In [16]:
number_of_images = len(df.image_name.unique())
print('The dataset contains {} images'.format(number_of_images))


The dataset contains 31783 images


In [19]:
df.image_name.value_counts().unique()

AttributeError: 'numpy.ndarray' object has no attribute 'value'

Every image contains the same amount of captions and thus, should be clean in this area.